In [13]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from konlpy.tag import Okt

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [14]:
movie_df = pd.read_csv( 'movie_reviews.csv' )

In [15]:
movie_df.shape

(384, 4)

In [16]:
movie_df.drop( movie_df.columns[ [ 0 ] ], axis = 'columns' )

,title,story,genre
0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마
1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마
2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험
3,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리
4,포드 V 페라리,1960년대 매출 감소에 빠진 포드는 판매 활로를 찾기 위해스포츠카 레이스를 장악한...,액션
...,...,...,...
379,바그다드 카페 : 디렉터스컷,황량한 사막 한가운데 자리 잡은 초라한 바그다드 카페 커피머신은 고장난지 오래고 먼...,코미디
380,론 서바이버,2005년 6월 28일 아프가니스탄에서 복무중인 네이비씰 대원 마커스 마이클 대니 ...,액션
381,국제시장,1950년대 한국전쟁 이후로부터 현재에 이르기까지 격변의 시대를 관통하며 살아온 우...,드라마
382,말레나,2차 대전이 한창인 햇빛 찬란한 지중해의 작은 마을 매혹적인 말레나 걸어갈 때면 어...,드라마


### 장르 코드 변수 추가

In [17]:
movie_genres = movie_df[ 'genre' ].unique().tolist()
movie_genres

['드라마',
 '모험',
 '다큐멘터리',
 '액션',
 'SF',
 '애니메이션',
 '범죄',
 '멜로/로맨스',
 '전쟁',
 '가족',
 '판타지',
 '코미디',
 '공연실황',
 '한국',
 '서부',
 '뮤지컬',
 '공포',
 '미스터리']

In [18]:
genre_length = len( movie_genres )

In [19]:
genre_code_dict = {}
index = 1
for genre in movie_genres:
    genre_code_dict[ genre ] = index
    index += 1
genre_code_dict

{'드라마': 1,
 '모험': 2,
 '다큐멘터리': 3,
 '액션': 4,
 'SF': 5,
 '애니메이션': 6,
 '범죄': 7,
 '멜로/로맨스': 8,
 '전쟁': 9,
 '가족': 10,
 '판타지': 11,
 '코미디': 12,
 '공연실황': 13,
 '한국': 14,
 '서부': 15,
 '뮤지컬': 16,
 '공포': 17,
 '미스터리': 18}

In [20]:
movie_df[ 'genre_code' ] = movie_df[ 'genre' ].map( lambda x : genre_code_dict[ x ] )

In [21]:
movie_df.head()

,Unnamed: 0,title,story,genre,genre_code
0,0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마,1
1,1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마,1
2,2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험,2
3,3,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리,3
4,4,포드 V 페라리,1960년대 매출 감소에 빠진 포드는 판매 활로를 찾기 위해스포츠카 레이스를 장악한...,액션,4


In [22]:
movie_df.shape

(384, 5)

### 학습에 사용할 변수에 대한 리스트 생성

In [23]:
movie_storys = np.array( movie_df[ 'story' ].values ) 
movie_genres = np.array( movie_df[ 'genre' ].values ) 
movie_genre_codes = np.array( movie_df[ 'genre_code' ].values )

In [24]:
print( '훈련용 샘플의 개수 : {}'.format( len( movie_storys ) ) )

훈련용 샘플의 개수 : 384


In [27]:
print( '총 장르의 개수 : {}'.format( len( genre_code_dict ) ) )

총 장르의 개수 : 18


In [30]:
movie_genres[ 0 ]

'드라마'

In [31]:
print('첫번째 샘플의 장르 : {}'.format( genre_code_dict[ movie_genres[ 0 ] ] ) )

첫번째 샘플의 장르 : 1


In [32]:
movie_storys.ndim, movie_storys.shape

(1, (384,))

In [33]:
movie_genres.ndim, movie_genres.shape

(1, (384,))

In [34]:
movie_genre_codes.ndim, movie_genre_codes.shape

(1, (384,))

In [35]:
movie_storys[ :3 ]

array(['1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아함 그 자체인천재 피아니스트 돈 셜리마허샬라 알리 박사의 운전기사 면접을 보게 된다백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사생각 행동 말투 취향까지 달라도 너무 다른 두 사람은그들을 위한 여행안내서 그린북에 의존해 특별한 남부 투어를 시작하는데',
       '나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 12살 소년 자인으로부터',
       '귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다시 시작된 견생 2회차 아니 3회차1등 경찰견 엘리에서 찰떡같이 마음을 알아주는 소울메이트 티노까지다시 태어날 때마다 성별과 생김새 직업에 이름도 바뀌지만여전히 영혼만은 사랑 충만 애교 충만 주인바라기 베일리어느덧 견생 4회차 방랑견이 되어 떠돌던 베일리는마침내 자신이 돌아온 진짜 이유를 깨닫고 어딘가로 달려가기 시작하는데'],
      dtype=object)

In [36]:
movie_genres[ :3 ]

array(['드라마', '드라마', '모험'], dtype=object)

In [37]:
movie_genre_codes[ :3 ]

array([1, 1, 2])

### 형태소 분석 / 불용어 처리

In [38]:
okt = Okt()

In [39]:
stop_words = []

with open( 'stopwords.txt', 'r', encoding = 'utf-8' ) as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append( line.rstrip( '\n' ) )

print( stop_words[ : 100 ] )

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다']


In [40]:
# 정제와 단어 토큰화
vocab = {} # 파이썬의 dictionary 자료형
storys = []

for story in movie_storys:
    sentence = okt.nouns( story ) # 단어 토큰화를 수행합니다.
    result = []

    for word in sentence: 
        if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
            result.append( word )
            if word not in vocab:
                vocab[ word ] = 0 
            vocab[ word ] += 1
    storys.append( result ) 
print( storys )

[['미국', '입담', '주먹', '토니', '발레', '롱', '비고', '모텐슨는', '교양', '자체', '천재', '피아니스트', '돈', '셜리마허샬', '알리', '박사', '운전기사', '면접', '백악관', '초청', '미국', '전역', '콘서트', '요청', '명성', '돈', '셜리', '미국', '남부', '투어', '공연', '결심', '투어', '기간', '보디가드', '겸', '운전기사', '토니', '고용', '인생', '토니', '발레', '롱', '교양', '기품', '돈', '셜리', '박사', '생각', '행동', '말투', '취향', '달라', '두', '사람', '위', '여행안내서', '그린북', '의존', '남부', '투어', '시작'], ['세상', '부모님', '출생', '기록', '어쩌면', '살', '소년', '자인'], ['소년', '단짝', '반려견', '베', '일리', '생', '마감', '눈', '다시', '시작', '견생', '경찰견', '엘리', '찰떡', '마음', '소울메이트', '티노', '다시', '성별', '생김새', '직업', '이름', '영혼', '사랑', '충만', '애교', '충만', '주인', '베', '일리', '견생', '방랑견', '베', '일리', '마침내', '진짜', '이유', '어딘가', '시작'], ['일본', '인종차별', '문제', '영상', '후', '우익', '공격', '대상', '일본', '미국인', '유튜버', '미키', '데', '자키', '일본군', '위안부', '관', '기사', '기자', '우익', '인신공격', '문제', '호기심', '안고', '이야기', '전하', '주장', '반격', '인물', '비밀', '발견', '숨', '틈', '전쟁', '시작'], ['매출', '감소', '포드', '판매', '활', '찾기', '위해', '스포츠카', '레이스', '장악', '절대', '위', '페라리', '인수', '합병', '추진', '막대', 

### 정수 인코딩 / 원-홧 인코딩

In [41]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts( storys )

In [42]:
print( tokenizer.word_index )

{'수': 1, '위해': 2, '시작': 3, '분': 4, '사랑': 5, '날': 6, '사람': 7, '알': 8, '그녀': 9, '세상': 10, '다시': 11, '아버지': 12, '말': 13, '친구': 14, '속': 15, '모든': 16, '곳': 17, '후': 18, '세계': 19, '두': 20, '집': 21, '발견': 22, '마음': 23, '살': 24, '가족': 25, '엄마': 26, '중': 27, '위': 28, '마을': 29, '못': 30, '채': 31, '인간': 32, '눈': 33, '사건': 34, '사실': 35, '전': 36, '아들': 37, '여자': 38, '남자': 39, '마지막': 40, '소녀': 41, '뒤': 42, '한편': 43, '이제': 44, '최고': 45, '향': 46, '죽음': 47, '온': 48, '삶': 49, '전쟁': 50, '가장': 51, '끝': 52, '앞': 53, '돈': 54, '운명': 55, '서로': 56, '꿈': 57, '딸': 58, '마침내': 59, '학교': 60, '모습': 61, '생각': 62, '소년': 63, '존': 64, '산': 65, '비밀': 66, '대한': 67, '위기': 68, '더': 69, '순간': 70, '미국': 71, '때문': 72, '인생': 73, '사이': 74, '도움': 75, '이름': 76, '이야기': 77, '아내': 78, '안': 79, '아빠': 80, '간다': 81, '힘': 82, '점점': 83, '길': 84, '우주': 85, '명의': 86, '아시': 87, '타카': 88, '생활': 89, '스': 90, '단': 91, '기억': 92, '게임': 93, '상황': 94, '이자': 95, '세': 96, '밤': 97, '목숨': 98, '해리': 99, '피터': 100, '미래': 101, '로봇': 102, '몸': 103, '작전': 1

In [43]:
print( tokenizer.word_counts )

OrderedDict([('미국', 43), ('입담', 3), ('주먹', 7), ('토니', 15), ('발레', 14), ('롱', 2), ('비고', 1), ('모텐슨는', 1), ('교양', 4), ('자체', 7), ('천재', 25), ('피아니스트', 7), ('돈', 50), ('셜리마허샬', 1), ('알리', 2), ('박사', 26), ('운전기사', 3), ('면접', 5), ('백악관', 1), ('초청', 2), ('전역', 4), ('콘서트', 8), ('요청', 12), ('명성', 5), ('셜리', 2), ('남부', 4), ('투어', 14), ('공연', 20), ('결심', 30), ('기간', 7), ('보디가드', 7), ('겸', 1), ('고용', 8), ('인생', 42), ('기품', 1), ('생각', 47), ('행동', 13), ('말투', 3), ('취향', 2), ('달라', 13), ('두', 86), ('사람', 162), ('위', 75), ('여행안내서', 1), ('그린북', 1), ('의존', 1), ('시작', 211), ('세상', 108), ('부모님', 10), ('출생', 1), ('기록', 12), ('어쩌면', 1), ('살', 81), ('소년', 47), ('자인', 1), ('단짝', 3), ('반려견', 1), ('베', 14), ('일리', 12), ('생', 17), ('마감', 3), ('눈', 68), ('다시', 103), ('견생', 3), ('경찰견', 1), ('엘리', 2), ('찰떡', 1), ('마음', 82), ('소울메이트', 2), ('티노', 1), ('성별', 1), ('생김새', 1), ('직업', 9), ('이름', 41), ('영혼', 16), ('사랑', 184), ('충만', 3), ('애교', 1), ('주인', 19), ('방랑견', 1), ('마침내', 48), ('진짜', 29), ('이유', 25), ('어딘가', 6), ('

In [61]:
max_words = 1000
num_classes = len( genre_code_dict )
tokenizer = Tokenizer( num_words = max_words ) # 상위 1000개 단어만 사용
tokenizer.fit_on_texts( storys )

In [63]:
storys_X = tokenizer.texts_to_matrix( storys, mode = 'binary' )

In [64]:
storys_X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [46]:
storys_integer = tokenizer.texts_to_sequences( storys )
storys_integer

[[71,
  863,
  310,
  345,
  864,
  157,
  865,
  54,
  150,
  71,
  736,
  445,
  54,
  71,
  346,
  204,
  121,
  346,
  866,
  867,
  310,
  737,
  73,
  310,
  345,
  54,
  150,
  62,
  388,
  389,
  20,
  7,
  28,
  346,
  3],
 [10, 559, 446, 24, 63],
 [63,
  347,
  447,
  258,
  33,
  11,
  3,
  23,
  11,
  633,
  76,
  287,
  5,
  216,
  347,
  447,
  347,
  447,
  59,
  125,
  158,
  3],
 [185,
  448,
  738,
  18,
  151,
  390,
  185,
  205,
  391,
  739,
  448,
  392,
  77,
  634,
  506,
  66,
  22,
  449,
  740,
  50,
  3],
 [635,
  194,
  2,
  868,
  450,
  28,
  186,
  869,
  348,
  635,
  130,
  507,
  20,
  39,
  19,
  870,
  741,
  451,
  635,
  636,
  2,
  393,
  870,
  737,
  742,
  311,
  45,
  90,
  637,
  635,
  871,
  7,
  28,
  3],
 [349,
  50,
  638,
  15,
  452,
  24,
  41,
  22,
  50,
  350,
  259,
  15,
  13,
  41,
  41,
  560,
  76,
  453,
  46,
  56,
  1,
  51,
  117,
  20,
  7,
  394,
  508,
  52,
  2,
  509,
  15,
  510,
  454,
  36,
  19,
  288,
  77],
 [

In [49]:
story_length = 0
for story in storys_integer:
    if len( story ) > story_length:
        story_length = len( story )
story_length

445

In [50]:
storys_integer = pad_sequences( storys_integer, maxlen = story_length, padding = 'post' )

In [51]:
storys_integer[ 0 ]

array([ 71, 863, 310, 345, 864, 157, 865,  54, 150,  71, 736, 445,  54,
        71, 346, 204, 121, 346, 866, 867, 310, 737,  73, 310, 345,  54,
       150,  62, 388, 389,  20,   7,  28, 346,   3,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

###  훈련 데이터와 테스트 데이터 분리

In [65]:
( X_train, X_test, y_train, y_test ) = train_test_split( storys_X, movie_genre_codes, train_size = 0.8, random_state = 2)

In [66]:
print( X_train[ :5 ] ) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [69]:
X_train.shape

(307, 1000)

y_train = to_categorical( y_train )
y_test = to_categorical( y_test )

In [68]:
print( y_train[ :5 ] )
print( y_test[ :5 ] )

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


### 소프트맥스 회귀

In [70]:
model=Sequential()

model.add( Dense( genre_length, input_dim = 307, activation = 'softmax' ) )
sgd = optimizers.SGD( lr = 0.01 )
model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )
history = model.fit( X_train, y_train, batch_size = 1, epochs = 200, validation_data = ( X_test, y_test ) )

ValueError: Error when checking input: expected dense_2_input to have shape (307,) but got array with shape (1000,)